In [ ]:
import pandas as pd
df = pd.read_csv("ground_truth_subjects.csv")


# Read CSV for training dataset
# Array file names
(df.head(1))


# ground_truth -> file_name -> peizo signals

# ground_truth -> file_name -> breathrate

# ground_truth -> file_name -> heartrate

# ground_truth -> file_name -> jpeaks

# ground_truth -> file_name -> movementvalue




# Visualize (Plot graph) data

# Pattern / Algo
# Linear regression to get breathrate and heartrate and jpeaks (continuous)
####



# Classfier movementvalues
####